In [1]:
import numpy as np
from itertools import product
from tqdm import tqdm
from dataclasses import dataclass, field
from matplotlib import pyplot as plt


def generate_data(P: int, N: int):
    xi = np.random.normal(loc=0, scale=1, size=(P, N))
    S = np.random.choice([1, -1], P)
    return xi, S

In [150]:
@dataclass
class Perceptron:
    xi: np.ndarray
    S: np.ndarray
    n_epoch: int = 1
    P: int = 0
    N: int = 0
    c: float = 0
    w: np.ndarray = field(init=False)
    x: np.ndarray = field(init=False) # embedding strength
    hebbians: np.ndarray = field(init=False)

    def __post_init__(self):
        self.P = self.xi.shape[0]
        self.N = self.xi.shape[1]
        self.w = np.zeros(self.N)
        self.x = np.zeros(self.P)

    def solution_found(self) -> bool:
        zero_update_count = sum(1 for mu in range(self.P) if np.dot(self.w, self.hebbians[mu]) > 0.0)
        return zero_update_count == self.P

    def trained(self):
        t = 0

        for n in range(self.n_epoch):
            self.hebbians = np.multiply(self.xi.T, self.S).T
            for mu in range(self.P):
                if self.solution_found():
                    return True
                t += 1
                E_mu = np.dot(self.w.T, self.hebbians[mu])
                if E_mu <= self.c:
                    self.w += self.xi[mu] * self.S[mu] / self.N
                    self.x[mu] += 1

        return False

    def get_embedding_strength(self):
        return self.x

In [135]:
P = 15
N = 20
xi, S = generate_data(P, N)
print(f"xi:\n{xi}")
print(f"S:\n{S}")

xi:
[[ 3.27592615e-01 -1.18536243e+00 -1.21184072e+00  1.14228461e+00
  -1.04641531e+00  1.41947573e+00  3.24226023e-01 -4.63982655e-01
  -8.24118963e-01  2.35731314e-01  5.22581039e-02 -4.28440288e-01
  -1.71849375e+00  5.53209215e-02  1.25497315e+00  9.07427287e-01
   1.37326161e-01  1.32459876e-01  2.37895398e-01 -4.63712004e-04]
 [ 5.39551243e-01 -5.40985075e-01  6.70584342e-01  1.35542925e+00
   1.74750647e+00 -1.31090996e+00  6.45404037e-01  1.39339117e-01
   5.59993686e-02 -2.35816619e-01  8.52185461e-01  1.07329155e+00
   1.01939006e-01  2.13404779e+00 -4.10467811e-01 -3.16075296e-01
  -3.86130895e-01  2.95052279e+00 -7.38200480e-01  8.78402280e-01]
 [-9.00638601e-01 -8.42237746e-01 -1.30345890e+00 -2.19608102e+00
  -6.02173135e-01  3.29441130e-01 -1.36203906e+00  1.67944880e+00
   8.00167587e-01  6.05929599e-01 -4.07771620e-03  3.43879368e-01
  -5.72692113e-01 -5.02426489e-01  6.50782455e-01  1.53880536e+00
   6.65646619e-01  9.20601602e-01 -8.33404818e-01 -4.85238914e-02]
 [-

In [130]:
def run_experiment(N, alpha_values, n_D, n_epoch, c=0.0):
    Q_ls = []
    x_values = []

    for alpha in alpha_values:
        P = int(alpha * N)
        print(f"P: {P}, N: {N}")
        success_count = 0

        for _ in range(n_D):
            xi, S = generate_data(P, N)
            p = Perceptron(xi, S, n_epoch, c)
            if (p.trained()):
                success_count += 1
                x_values.extend(p.get_embedding_strength())

        Q_ls.append(success_count / n_D)
        print(f"success_count/n_D: {success_count}/{n_D}")
        print("\n")

    return Q_ls, x_values

In [ ]:
N = 20
n_D = 50
n_max = 100
alpha_values = [1.0]

Q_ls, x_values = run_experiment(N, alpha_values, n_D, n_max)
print(f"Q_ls: {Q_ls}")
print(f"x_values.shape: {len(x_values)}")

In [ ]:
n_D = 50
n_max = 100
alpha_values = np.arange(0.75, 3.25, 0.25)

for N in [20, 40]:
    Q_ls, _ = run_experiment(N, alpha_values, n_D, n_max)
    plt.plot(alpha_values, Q_ls, marker='o', label=f'N={N}')

plt.xlabel('α = P/N')
plt.ylabel('$Q_{l.s.}$')
plt.legend()
plt.show()

In [ ]:
n_D = 50
n_max = 100
alpha_values = np.arange(0.75, 3.25, 0.25)

for N in [20, 40]:
      Q_ls, x_values = run_experiment(N, alpha_values, n_D, n_max)
      plt.figure(figsize=(12, 6))
      plt.subplot(1, 2, 1)
      plt.plot(alpha_values, Q_ls, marker='o', label=f'N={N}')
      plt.xlabel('α = P/N')
      plt.ylabel('$Q_{l.s.}$')
      plt.legend()
      plt.subplot(1, 2, 2)
      plt.hist(x_values, bins=range(1, n_max+2), edgecolor='black', alpha=0.7)
      plt.xlabel('Embedding Strengths $x^µ$')
      plt.ylabel('Frequency')
      plt.tight_layout()
      plt.show()

In [ ]:
n_D = 50
n_max = 100
c_values = [0, 0.5, 1.0]
alpha_values = np.arange(0.75, 3.25, 0.25)

for N in [20, 40]:
    for c in c_values:
        Q_ls, _ = run_experiment(N, alpha_values, n_D, n_max, c)
        print(f"Q_ls: {Q_ls}")
        plt.plot(alpha_values, Q_ls, marker='o', label=f'N={N}, c={c}')
    plt.xlabel('α = P/N')
    plt.ylabel('$Q_{l.s.}$')
    plt.legend()
    plt.show()